In [23]:
import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('.'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


from sklearn.metrics.pairwise import cosine_similarity

./RecommenderSystem.ipynb
./.git/config
./.git/HEAD
./.git/description
./.git/index
./.git/packed-refs
./.git/COMMIT_EDITMSG
./.git/FETCH_HEAD
./.git/objects/04/dddb75984d0187623acd34c617572da8796ab8
./.git/objects/ae/3c19dc95c27ab36fce9fb4bf0c9abd71a1c603
./.git/objects/f4/b9e637e8e0cfe5d90115dc14e95b1c0d226fb3
./.git/objects/e3/a6820dcb30d433634a81b1f1a9284a2292afe7
./.git/objects/c6/b7831384361daec0972507ef5f38c9bb18223c
./.git/objects/pack/pack-ec4e5e329ba013a02c0664c02a9f89cadf33ad57.pack
./.git/objects/pack/pack-ec4e5e329ba013a02c0664c02a9f89cadf33ad57.idx
./.git/objects/2a/23ce30bbb55de031e1b33a5bb1d93a0ecd7d80
./.git/objects/9a/0f6f5432f2f9d8564c3559fa2fe71f5663e1b1
./.git/objects/3f/5f0b61c3d037ec3f02a74f24017c7b038eb0dc
./.git/objects/d5/07e9338fa83ce86d1a438914a8f0a8f98dddc7
./.git/info/exclude
./.git/logs/HEAD
./.git/logs/refs/heads/RecommenderModel
./.git/logs/refs/heads/main
./.git/logs/refs/remotes/origin/HEAD
./.git/logs/refs/remotes/origin/RecommenderModel
./.git/hooks

# Load Data

In [24]:
books = pd.read_csv('/Users/thabochesane/Desktop/path2/books_data.csv')
reviews = pd.read_csv('/Users/thabochesane/Desktop/path2/Books_rating.csv')





In [25]:
books.shape, reviews.shape

((212404, 10), (3000000, 10))

# Filtering

In [26]:
reviews = reviews[['Id','User_id','review/score','Title']]

In [27]:
reviews.rename(columns={'Id' : 'BookId', 'User_id' : 'UserId', 'review/score' : 'Review' , 'Title' : 'BookTitle' }, inplace= True)

In [28]:
reviews.head(100)

,BookId,UserId,Review,BookTitle
0,1882931173,AVCGYZL8FQQTD,4.0,Its Only Art If Its Well Hung!
1,0826414346,A30TK6U7DNS82R,5.0,Dr. Seuss: American Icon
2,0826414346,A3UH4UZ4RSVO82,5.0,Dr. Seuss: American Icon
3,0826414346,A2MVUWT453QH61,4.0,Dr. Seuss: American Icon
4,0826414346,A22X4XUPKF66MR,4.0,Dr. Seuss: American Icon
...,...,...,...,...
95,B000NKGYMK,A2UMP9TJTJ6A6B,1.0,Alaska Sourdough
96,B000NKGYMK,AC2TK7NHKB5C0,5.0,Alaska Sourdough
97,B000NKGYMK,A22T74YNRM8NTK,5.0,Alaska Sourdough
98,B000NKGYMK,A2E0GB5QZRX2JZ,5.0,Alaska Sourdough


# Get number of reviews per book

In [29]:
numberofReviews = reviews.groupby('BookTitle').count()['Review'].reset_index()
numberofReviews.rename(columns={'Reviews' : 'NumberOfReviews'}, inplace= True)

In [30]:
numberofReviews.head()

,BookTitle,Review
0,""" Film technique, "" and, "" Film acting """,2
1,""" We'll Always Have Paris"": The Definitive Gui...",2
2,"""... And Poetry is Born ..."" Russian Classical...",1
3,"""A Titanic hero"" Thomas Andrews, shipbuilder",8
4,"""A Truthful Impression of the Country"": Britis...",1


# Filter the users and books.

In [31]:
userId = reviews.groupby('UserId').count()['Review'] > 200 
activeUsers = userId[userId].index
cleanedReviews = reviews[reviews['UserId'].isin(activeUsers)]
cleanedReviews

,BookId,UserId,Review,BookTitle
1,0826414346,A30TK6U7DNS82R,5.0,Dr. Seuss: American Icon
6,0826414346,A14OJS0VWMOSWO,5.0,Dr. Seuss: American Icon
59,0963923080,A3NIQK6ZLYEP1L,4.0,Rising Sons and Daughters: Life Among Japan's ...
140,0789480662,A281NPSIMI1C2R,5.0,Eyewitness Travel Guide to Europe
176,1884734766,A14OJS0VWMOSWO,5.0,Dealing With Disappointment: Helping Kids Cope...
...,...,...,...,...
2999935,0786182431,A2CR57GAJKNWVV,4.0,Very Bad Deaths: Library Edition
2999937,0786182431,A1I2O9Y3X3HXLS,5.0,Very Bad Deaths: Library Edition
2999949,0786182431,AFVQZQ8PW0L,5.0,Very Bad Deaths: Library Edition
2999960,0679417052,A9Q28YTLYREO7,5.0,My Life


In [32]:
x = cleanedReviews.groupby('BookTitle').count()['Review'] >= 50
popularBooks = x[x].index

reviewsDataFrame = cleanedReviews[cleanedReviews['BookTitle'].isin(popularBooks)]
reviewsDataFrame

,BookId,UserId,Review,BookTitle
1747,B000N6DDJQ,AUM3YMZ0YRJE0,5.0,The Scarlet Letter A Romance
1807,B000N6DDJQ,AMKZHBOK7VMQR,5.0,The Scarlet Letter A Romance
1820,B000N6DDJQ,AWLFVCT9128JV,4.0,The Scarlet Letter A Romance
1829,B000N6DDJQ,ABN5K7K1TM1QA,5.0,The Scarlet Letter A Romance
1830,B000N6DDJQ,A3IKBHODOTYYHM,4.0,The Scarlet Letter A Romance
...,...,...,...,...
2999633,B000P91JYW,A1D2C0WDCSHUWZ,5.0,Jane Eyre
2999638,B000P91JYW,AHUT55E980RDR,4.0,Jane Eyre
2999641,B000P91JYW,A1T17LMQABMBN5,5.0,Jane Eyre
2999646,B000P91JYW,AJQ1S39GZBKUG,5.0,Jane Eyre


In [33]:
reviewsDataFrame.to_csv('/Users/thabochesane/Desktop/path2/reviews.csv')

In [34]:
reviewsDataFrame.head()

,BookId,UserId,Review,BookTitle
1747,B000N6DDJQ,AUM3YMZ0YRJE0,5.0,The Scarlet Letter A Romance
1807,B000N6DDJQ,AMKZHBOK7VMQR,5.0,The Scarlet Letter A Romance
1820,B000N6DDJQ,AWLFVCT9128JV,4.0,The Scarlet Letter A Romance
1829,B000N6DDJQ,ABN5K7K1TM1QA,5.0,The Scarlet Letter A Romance
1830,B000N6DDJQ,A3IKBHODOTYYHM,4.0,The Scarlet Letter A Romance


# Check for Null values

In [35]:
nullColumns = reviewsDataFrame.isnull().any()
print(nullColumns)

BookId       False
UserId       False
Review       False
BookTitle    False
dtype: bool


# Check for Not a number (NaN) values

In [36]:
x = reviewsDataFrame.pivot_table(index='UserId', columns='BookTitle', values= 'Review')
x

BookTitle,1984,A Christmas Carol (Classic Fiction),A Connecticut Yankee in King Arthur's Court,A Farewell to Arms,A Fine Balance,A Journey to the Center of the Earth,A Portrait of the Artist As a Young Man,A Portrait of the Artist as a Young Man,A Princess of Mars,A Tree Grows in Brooklyn,...,"Ulysses,",Ulysses.,Wuthering Heights,Wuthering Heights (College classics in English),Wuthering Heights (New Windmill),Wuthering Heights (Penguin Audiobooks),Wuthering Heights (Riverside editions),Wuthering Heights (Signet classics),Wuthering Heights (The Franklin Library),Wuthering Heights.
UserId,,,,,,,,,,,,,,,,,,,,,
A106016KSI0YQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A10T0OW97SFBB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0
A114YQ7ZT9Y1W5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A11DCTGTPS7M0C,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,...,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A126KX6FVI4T66,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AV74NYPSKHXBU,4.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,5.0,...,NaN,NaN,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
AWLFVCT9128JV,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AX54G5AL870Q8,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
nan_counts = x.isna().sum()
print(nan_counts)

BookTitle
1984                                           246
A Christmas Carol (Classic Fiction)            273
A Connecticut Yankee in King Arthur's Court    272
A Farewell to Arms                             258
A Fine Balance                                 280
                                              ... 
Wuthering Heights (Penguin Audiobooks)         258
Wuthering Heights (Riverside editions)         258
Wuthering Heights (Signet classics)            258
Wuthering Heights (The Franklin Library)       258
Wuthering Heights.                             258
Length: 176, dtype: int64


In [38]:
x.fillna(0, inplace=True)
x

BookTitle,1984,A Christmas Carol (Classic Fiction),A Connecticut Yankee in King Arthur's Court,A Farewell to Arms,A Fine Balance,A Journey to the Center of the Earth,A Portrait of the Artist As a Young Man,A Portrait of the Artist as a Young Man,A Princess of Mars,A Tree Grows in Brooklyn,...,"Ulysses,",Ulysses.,Wuthering Heights,Wuthering Heights (College classics in English),Wuthering Heights (New Windmill),Wuthering Heights (Penguin Audiobooks),Wuthering Heights (Riverside editions),Wuthering Heights (Signet classics),Wuthering Heights (The Franklin Library),Wuthering Heights.
UserId,,,,,,,,,,,,,,,,,,,,,
A106016KSI0YQ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A10T0OW97SFBB,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0
A114YQ7ZT9Y1W5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A11DCTGTPS7M0C,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A126KX6FVI4T66,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AV74NYPSKHXBU,4.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,...,0.0,0.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
AWLFVCT9128JV,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AX54G5AL870Q8,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# simlarity analysis

In [39]:
simlarityResults = cosine_similarity(x)

In [40]:
simlarityResults

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.07151985, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.07151985, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.22086305,
        0.18586518],
       [0.        , 0.        , 0.        , ..., 0.22086305, 1.        ,
        0.42077019],
       [0.        , 0.        , 0.        , ..., 0.18586518, 0.42077019,
        1.        ]])

In [47]:
def recommendBooks(userId):
    i = np.where(x.index == userId)[0][0]
    similarBooks = sorted(list(enumerate(simlarityResults[0])), key=lambda x:x[1], reverse=True)[1:4]

    userData = []

    for n in similarBooks:
        df = reviews[reviews['UserId'] == x.index[n[0]]].sort_values(by = 'Review', ascending = False)

        listOfUserbooks = list(reviews[reviews['UserId'] == x.index[i]]['BookTitle'])
        df = df[~df['BookTitle'].isin(listOfUserbooks)]
        df = df[df['Review'].isin([4.0,5.0])]
        userData.extend(list(df.head()['BookTitle']))
        userData = [*set(userData)]
    
    return userData